In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import losses
import pandas as pd
import numpy as np
import sklearn
import scipy
pd.set_option('display.max_columns', 500)
from array import array
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

In [14]:
import matplotlib.pyplot as plt

In [2]:
#keeping this seperate to have access to the raw, unprocessed data as needed
data = pd.read_csv("/Users/elianasullivan/Documents/Minerva/last one/second semester/capstone/full clean data.csv")

In [3]:
#remove unnecessary columns
data = data.drop(data.columns[0], axis=1)
data = data.drop(data.columns[0], axis=1)
data=(data-data.min())/(data.max()-data.min())

In [4]:
data.head()

,ACEDEPRS,ACEDRINK,ACEDRUGS,ACEPRISN,ACEDIVRC,ACEPUNCH,ACEHURT,ACESWEAR,ACETOUCH,ACETTHEM,ACEHVSEX,DIABETE2,CVDINFR4,CVDCRHD4,CVDSTRK3,ASTHMA2,SMOKE100,VETERAN2,ADANXEV,ADDEPEV,@_BMI4CAT,DROCDY2_,@_RFBING4,@_SMOKER3,PHYSHLTH,MENTHLTH,QLREST2,EXERANY2,AGE,EDUCA,INCOME2,SEX,GENHLTH_1.0,GENHLTH_2.0,GENHLTH_3.0,GENHLTH_4.0,GENHLTH_5.0,@_PRACE_1.0,@_PRACE_2.0,@_PRACE_3.0,@_PRACE_4.0,@_PRACE_5.0,@_PRACE_6.0,@_STATE_11,@_STATE_15,@_STATE_32,@_STATE_50,@_STATE_55
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.00,0.0,1.0,0.000000,0.0,0.066667,0.142857,0.617284,0.6,0.714286,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.166667,0.0,0.000000,0.142857,0.370370,0.6,1.000000,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.33,0.0,1.0,0.133333,0.0,0.000000,0.142857,0.604938,1.0,1.000000,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.43,1.0,1.0,0.133333,0.0,0.933333,0.000000,0.271605,0.8,1.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.07,0.0,1.0,0.000000,0.0,0.000000,0.142857,0.567901,1.0,0.857143,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy

saved_models = []

#need to just put in the outcomes I want to do here
for outcome in ['DIABETE2','CVDINFR4','CVDCRHD4','CVDSTRK3','ASTHMA2','SMOKE100','ADANXEV','ADDEPEV','@_BMI4CAT','@_RFBING4', '@_SMOKER3']:

    seed = 7
    numpy.random.seed(seed)

    # split into input (X) and output (Y) variables
    #split the data up so that the ACEs and demographic data are the inputs
    X_data = data[['ACEDEPRS','ACEDRINK','ACEDRUGS','ACEPRISN','ACEDIVRC','ACEPUNCH','ACEHURT','ACESWEAR',\
                   'ACETOUCH','ACETTHEM','ACEHVSEX','VETERAN2','AGE','EDUCA',\
                   'INCOME2','SEX','@_PRACE_1.0','@_PRACE_2.0','@_PRACE_3.0','@_PRACE_4.0','@_PRACE_5.0',\
                   '@_PRACE_6.0']]

    #the binary variable for the outcome -- THIS CHANGES FOR EACH MODEL
    y_data = data[[outcome]]


    #creates a test set
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3)

    X = X_train
    Y = y_train

    # define 10-fold cross validation test harness
    #can increase n_splits and also the epochs
    kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)

    cvscores = []
    for train, test in kfold.split(X, Y):

        #create model
        classifier = Sequential()

        # Adding the input layer and the first hidden layer
        classifier.add(Dense(kernel_initializer="random_uniform", activation="relu", input_dim=len(list(X_train)), units=500))

        # Adding the second hidden layer
        classifier.add(Dense(units=100, activation="relu", kernel_initializer="random_uniform"))

        # # Adding the third hidden layer
        classifier.add(Dense(units=70, activation="relu", kernel_initializer="random_uniform"))

        # # Adding the fourth hidden layer
        classifier.add(Dense(units=40, activation="relu", kernel_initializer="random_uniform"))

        # # Adding the fifth hidden layer
        classifier.add(Dense(units=30, activation="relu", kernel_initializer="random_uniform"))

        # # Adding the sixth hidden layer
        classifier.add(Dense(units=20, activation="relu", kernel_initializer="random_uniform"))

        #sigmoid is crucial for binary results 
        classifier.add(Dense(units=1, activation="sigmoid", kernel_initializer="random_uniform"))

        # Compile model
        classifier.compile(optimizer = "adam", loss = losses.binary_crossentropy, metrics = ['accuracy'])

        # Fit the model
        #classifier.fit(np.array(X_train), np.array(y_train),epochs = 500, verbose=1, batch_size=32, callbacks = [])
        classifier.fit(X.iloc[train], Y.iloc[train], epochs=200, batch_size=10, verbose=0)

        # evaluate the model
        scores = classifier.evaluate(X.iloc[test], Y.iloc[test], verbose=1)

        print("%s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))

        cvscores.append(scores[1] * 100)

    print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))
    saved_models.append(classifier)
    

In [7]:
#I have 9 neural nets here
saved_models

In [26]:
# test that the models work
preds = saved_models[6].predict(X_train)

In [81]:
# save each of the models
# I just did it manually to have more control over naming
# serialize model to JSON
model_json = saved_models_new[1].to_json()
with open("smoker.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
saved_models_new[1].save_weights("smoker.h5")
print("Saved model to disk")

Saved model to disk


In [64]:
# open files
json_file = open('test.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("test.h5")
print("Loaded model from disk")

Loaded model from disk
